In [1]:
from torchvision import datasets, transforms
import torch
def calculate_mean_std(dataset):
    """calculate mean and standard deviation of a dataset"""
    mean = 0.0
    std = 0.0
    total_images = len(dataset)

    for img, _ in dataset:
        mean += img.mean()
        std += img.std()

    mean /= total_images
    std /= total_images

    return mean.item(), std.item()

# normalization transform
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# load training and test datasets with normalization
trainset = datasets.FashionMNIST(root='./data/', train=True, download=True, transform=transform)
valset = datasets.FashionMNIST(root='./data/', train=False, download=True, transform=transform)

# mean, std = calculate_mean_std(trainset)
# print(f'Before normalization: mean: {mean}, std: {std}')

# # normalization transform
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((mean,), (std,))])

# # load training and test datasets with normalization
# trainset = datasets.FashionMNIST(root='./data/', train=True, download=True, transform=transform)
# valset = datasets.FashionMNIST(root='./data/', train=False, download=True, transform=transform)

# mean, std = calculate_mean_std(trainset)
# print(f'After normalization: mean: {mean}, std: {std}')

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [2]:
import torch
import torch.nn as nn
from torch.optim import Adadelta
from torch.optim.lr_scheduler import StepLR
import sys
from pathlib import Path

utils_dir = Path('/home/gustaf/projects/deeplearning/utils')
sys.path.append(str(utils_dir))
from metrics import precision, recall, f1_score
from trainer import ModelTrainer


# model
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        # conv layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(8)  # batch norm after first conv
        self.bn2 = nn.BatchNorm2d(16)  # batch norm after second conv

        # size calculation
        # 28x28 → conv2: 28x28 → pool: 14x14
        # 14x14 → conv2: 14x14 → pool: 7x7
        # 7x7 → flatten: 7 * 7 * 16 = 784
        flattened_size = 784

        # fully connected layers
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(flattened_size, output_size)  # direct output layer

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # regularization

    def forward(self, x):
        # conv1 + batch norm + pooling
        x = self.pool(self.bn1(self.relu(self.conv1(x))))
        # conv2 + batch norm + pooling
        x = self.pool(self.bn2(self.relu(self.conv2(x))))

        # flatten and fully connected layer
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# initialize model
model = Net(input_size=28*28, output_size=10)
# initialize optimizer
optimizer = Adadelta(model.parameters(), lr=1, weight_decay=0.002)
# initialize scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# initialize trainer
trainer = ModelTrainer(
    model=model,
    device=device,
    loss_fn=nn.CrossEntropyLoss(),
    optimizer=optimizer,
    scheduler=scheduler,
    batch_size=512,
    verbose=True,
    early_stopping_patience=5,
    early_stopping_delta=1,
    metrics=[ModelTrainer.accuracy]
)

# train model
trained_model = trainer.train(
    training_set=trainset,
    val_set=valset,
    num_epochs=10
)

[Epoch 01] Train Loss: 0.6221 | Val Loss: 0.5488 | accuracy: 80.12%
Validation loss decreased (inf --> 0.5488). Saving model.
[Epoch 02] Train Loss: 0.3935 | Val Loss: 0.4188 | accuracy: 84.42%
EarlyStopping counter: 1 out of 5
[Epoch 03] Train Loss: 0.3513 | Val Loss: 0.3851 | accuracy: 86.19%
EarlyStopping counter: 2 out of 5
[Epoch 04] Train Loss: 0.3272 | Val Loss: 0.3369 | accuracy: 87.57%
EarlyStopping counter: 3 out of 5
[Epoch 05] Train Loss: 0.3144 | Val Loss: 0.3190 | accuracy: 88.57%
EarlyStopping counter: 4 out of 5
[Epoch 06] Train Loss: 0.3041 | Val Loss: 0.3042 | accuracy: 88.99%
EarlyStopping counter: 5 out of 5
🚨 Early stopping triggered.


KeyboardInterrupt: 

<Figure size 1500x600 with 0 Axes>

In [2]:
import sys
sys.path

['/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/gustaf/projects/deeplearning/venv/lib/python3.10/site-packages',
 '/tmp/tmp6wk5mg6e']

In [11]:
print(type(1e-3))

<class 'float'>


In [12]:
print(type(0.001))

<class 'float'>
